# Claim-based Question-Answer Hallucination Detection

Import necessary packages.

In [1]:
import time
from uqlm.longform.black_box import ClaimQAScorer
from uqlm import BlackBoxUQ

#### Load LLM and device

In [2]:
# from langchain_google_vertexai import ChatVertexAI
# llm = ChatVertexAI(model="gemini-1.5-flash")

from dotenv import load_dotenv, find_dotenv
from langchain_openai import AzureChatOpenAI

load_dotenv(find_dotenv())
llm = AzureChatOpenAI(
    deployment_name="gpt-4o-mini",
    openai_api_type="azure",
    openai_api_version="2024-02-15-preview",
    temperature=1,  # User to set temperature
)

In [3]:
import torch

# Set the torch device
if torch.cuda.is_available():  # NVIDIA GPU
    device = torch.device("cuda")
elif torch.backends.mps.is_available():  # macOS
    device = torch.device("mps")
else:
    device = torch.device("cpu")  # CPU
print(f"Using {device.type} device")

Using mps device


#### Setup Prompts and Black Box Scorer

In [4]:
prompts = [
    "write a paragraph about Paul McCartney",
    "write a paragraph about John Lennon"
]

In [5]:
bb_scorer = BlackBoxUQ(
    llm=llm,
    max_calls_per_min=500,  # set value to avoid rate limit error
    device=device,
    scorers=["exact_match"],
)

#### Claim-QA class

There are three methods that can be used to compute Claim-QA score.
- `generate_and_score`: If you only have prompts, call this method generate long response, decompose that response into factoids, then generate questions for each factoids, and compute question-level, factoid-level, and response-level scores.
- `score`: If you already generated long response, call this method
- `evaluate`: If you already have decomposed long responses into factoids, call this method

##### 1. `generate_and_score` method

In [6]:
claim_qa = ClaimQAScorer(llm=llm, black_box_scorers=["exact_match"], response_template="atomic", max_calls_per_min=500, num_questions=2)
start_time = time.time()
result = await claim_qa.generate_and_score(prompts=prompts)
print(f"Computation time: {time.time() - start_time} seconds")

Generated questions:  ['### What can you tell me about influential musicians from England?  \n### Who are some notable figures in the music industry known for their songwriting abilities?  ', "### What are some of the primary roles that Paul McCartney has taken on throughout his career?  \n### How would you describe Paul McCartney's contributions to the music industry?", "### What is one of Paul McCartney's primary roles in the music industry?  \n### How would you describe Paul McCartney's contributions to music?  ", "### What is Paul McCartney's most significant musical achievement?  \n### How is Paul McCartney recognized in the music industry?", '### What is the birth date of the famous musician discussed in the text?  \n### When did Paul McCartney enter the world, according to his biographical information?  ', "### What city did Paul McCartney grow up in?  \n### Where was Paul McCartney's birthplace?  ", "### What early experiences contributed to Paul McCartney's musical development

In [7]:
result.to_df()

,prompt,response,response_scores_exact_match,factoid_scores_exact_match
0,write a paragraph about Paul McCartney,"Paul McCartney is a renowned English musician,...",0.223077,"[0.0, 0.2, 0.5, 0.0, 1.0, 1.0, 0.0, 0.0, 0.1, ..."
1,write a paragraph about John Lennon,"John Lennon was an iconic British musician, so...",0.237037,"[0.2, 0.0, 0.3, 1.0, 0.1, 0.4, 0.5, 0.5, 0.4, ..."


#### 2. `score` method

In [8]:
claim_qa2 = ClaimQAScorer(llm=llm, black_box_scorers=["exact_match"], response_template="atomic", max_calls_per_min=500)
start_time = time.time()
result2 = await claim_qa2.score(prompts=claim_qa.prompts, responses=claim_qa.responses)
print(f"Computation time: {time.time() - start_time} seconds")

Generated questions:  ['### What are some notable accomplishments of Paul McCartney in the music industry?', "### What can you tell me about Paul McCartney's musical career and contributions?", '### What role did Paul McCartney play in the music industry?', '### What key contributions is Paul McCartney recognized for in the music industry?', '### When was Paul McCartney born?', '### Where was Paul McCartney born?', "### What early experiences contributed to Paul McCartney's development as a musician?", "### What impact did Paul McCartney's collaboration with another musician have on music history?", '### What are some of the notable achievements of Paul McCartney during his time with The Beatles?', '### What are some of the notable songs written by Paul McCartney during his time with The Beatles?', '### What are some notable songs that Paul McCartney wrote during his time with The Beatles?', '### What are some of the iconic songs associated with Paul McCartney and The Beatles?', '### W

In [9]:
result2.to_df()

,prompt,response,response_scores_exact_match,factoid_scores_exact_match
0,write a paragraph about Paul McCartney,"Paul McCartney is a renowned English musician,...",0.160000,"[0.0, 0.0, 0.2, 0.2, 1.0, 1.0, 0.0, 0.0, 0.0, ..."
1,write a paragraph about John Lennon,"John Lennon was an iconic British musician, so...",0.348148,"[1.0, 0.0, 0.0, 1.0, 0.4, 0.6, 1.0, 1.0, 1.0, ..."


#### 3. `evaluate` method

In [10]:
claim_qa3 = ClaimQAScorer(llm=llm, black_box_scorers=["exact_match"], response_template="atomic", max_calls_per_min=500)
start_time = time.time()
result3 = await claim_qa3.evaluate(prompts=claim_qa.prompts, responses=claim_qa.responses, factoids=claim_qa.factoids)
print(f"Computation time: {time.time() - start_time} seconds")

Generated questions:  ['### Who is a prominent figure in the music industry known for his work with a famous band and as a solo artist?', '### What role did Paul McCartney play in the music industry?', '### What role did Paul McCartney play in the music industry?', '### What are some of the most significant contributions of Paul McCartney to music history?', '### When was Paul McCartney born?', '### Where was Paul McCartney born?', '### What early experiences did Paul McCartney have that indicated his musical abilities?', '### What were some of the most famous collaborations in music history?', '### What are some notable songs that Paul McCartney contributed to during his time with The Beatles?', '### What impact did Paul McCartney have on the music industry through his work with The Beatles?', "### What are some notable aspects of Paul McCartney's role in The Beatles?", '### What band is Paul McCartney famously associated with from the 1960s?', '### When did the collaborative partners

In [11]:
result3.to_df()

,prompt,response,response_scores_exact_match,factoid_scores_exact_match
0,write a paragraph about Paul McCartney,"Paul McCartney is a renowned English musician,...",0.253846,"[1.0, 0.0, 0.4, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, ..."
1,write a paragraph about John Lennon,"John Lennon was an iconic British musician, so...",0.288889,"[1.0, 0.0, 0.0, 0.8, 0.0, 0.0, 1.0, 1.0, 1.0, ..."
